In [1]:
import threading as th
import random
import time
import numpy as np

## Programa 1

Implementa un programa que simule la carrera entre la liebre y la tortuga. En una pista de 70 metros, y avanzando con los valores definidos en la tabla del examen.

In [2]:
class Tortuga(th.Thread):
    

    def __init__(self):
        th.Thread.__init__(self)
        self.probabilidad = 0
        self.recorrido_tortuga = 1
        self.control = True
        
    # Para detener el ciclo while en run
    def stop(self):
        self.control = False
    
    def run(self):
        while (self.control):
            # Genero un número aleatorio en cada iteración y lo comparo con los valores
            # de la tabla para saber el comportamiento del animal
            self.probabilidad = random.randint(1,100)
            if self.probabilidad <= 50:
                self.recorrido_tortuga += 3
                if self.recorrido_tortuga > 70:
                    self.recorrido_tortuga = 70
                print("Posicion {}: {}T".format(self.recorrido_tortuga, " "*self.recorrido_tortuga))

            elif self.probabilidad <= 70:
                self.recorrido_tortuga -= 6
                if self.recorrido_tortuga < 1:
                    self.recorrido_tortuga = 1
                print("Posicion {}: {}T".format(self.recorrido_tortuga, " "*self.recorrido_tortuga))
            else:
                self.recorrido_tortuga += 1
                if self.recorrido_tortuga > 70:
                    self.recorrido_tortuga = 70
                print("Posicion {}: {}T".format(self.recorrido_tortuga, " "*self.recorrido_tortuga))
            time.sleep(1)

In [3]:
class Liebre(th.Thread):
   
    def __init__(self):
        th.Thread.__init__(self)
        self.probabilidad = 0
        self.recorrido_liebre = 1
        self.control = True
    
    # Para detener el ciclo while en run
    def stop(self):
        self.control = False
        
    def run(self):
        while (self.control):
            # Genero un número aleatorio en cada iteración y lo comparo con los valores
            # de la tabla para saber el comportamiento del animal
            self.probabilidad = random.randint(1,100)
            if self.probabilidad <= 20:
                self.recorrido_liebre = self.recorrido_liebre
                print("Posicion {}: {}L".format(self.recorrido_liebre, " "*self.recorrido_liebre))

            elif self.probabilidad <= 40:
                self.recorrido_liebre += 9
                if self.recorrido_liebre > 70:
                    self.recorrido_liebre = 70
                print("Posicion {}: {}L".format(self.recorrido_liebre, " "*self.recorrido_liebre))

            elif self.probabilidad <= 50:
                self.recorrido_liebre -= 12
                if self.recorrido_liebre < 1:
                    self.recorrido_liebre = 1
                print("Posicion {}: {}L".format(self.recorrido_liebre, " "*self.recorrido_liebre))

            elif self.probabilidad <= 80:
                self.recorrido_liebre += 1
                if self.recorrido_liebre > 70:
                    self.recorrido_liebre = 70
                print("Posicion {}: {}L".format(self.recorrido_liebre, " "*self.recorrido_liebre))
            else:
                self.recorrido_liebre -= 2
                if self.recorrido_liebre < 1:
                    self.recorrido_liebre = 1
                print("Posicion {}: {}L".format(self.recorrido_liebre, " "*self.recorrido_liebre))

            time.sleep(1)

In [4]:
# Cada animal tendra su propio recorrido y sera incrementado o decrementado de manera independiente
lock = th.Lock()

tortuga = Tortuga()
liebre = Liebre()

tortuga.start()
liebre.start()

# Con esta condición definimos cuando hay un empate o alguien pierde

while True:
    if liebre.recorrido_liebre == 70 and tortuga.recorrido_tortuga == 70:
        print("Empate")
        tortuga.stop()
        liebre.stop()
        tortuga.join()
        liebre.join()
        break
        
    if liebre.recorrido_liebre == 70:
        print("Liebre ganó")
        tortuga.stop()
        liebre.stop()
        tortuga.join()
        liebre.join()
        break
        
    if tortuga.recorrido_tortuga == 70:
        print("Tortuga ganó")
        tortuga.stop()
        liebre.stop()
        tortuga.join()
        liebre.join()
        break

Posicion 1:  TPosicion 10:           L

Posicion 8:         L
Posicion 4:     T
Posicion 8:         LPosicion 1:  T

Posicion 17:                  LPosicion 4:     T

Posicion 15:                L
Posicion 5:      T
Posicion 13:              L
Posicion 6:       T
Posicion 22:                       LPosicion 9:          T

Posicion 22:                       LPosicion 12:             T

Posicion 22:                       LPosicion 15:                T

Posicion 18:                   T
Posicion 31:                                L
Posicion 19:                    TPosicion 19:                    L

Posicion 28:                             L
Posicion 22:                       T
Posicion 25:                          TPosicion 28:                             L

Posicion 26:                           LPosicion 26:                           T

Posicion 27:                            TPosicion 27:                            L

Posicion 36:                                     LPosicion 30:       

## Programa 2

Implementar una multiplicación de matrices secuencial, y paralelo utilizando 4 threads.

In [5]:
def multiplicacion_matrices(A, B):
    
    C = []
    
    Am = len(A) 
    An = len(A[0])
    
    Bm = len(B)
    Bn = len(B[0])
    
    if An != Bm:
        return False
    
    for k in range(Am):
        fila_C = []
        
        
        for i in range(Bn):
            sumador = 0
            for j in range(Bm):
                sumador += A[k,j] * B[j,i]
            
            fila_C.append(sumador)
        C.append(fila_C)
    
    return C

In [6]:
A = np.random.randint(10, size=(50, 50))
B = np.random.randint(10, size=(50, 50))
 
t = time.time()
C = multiplicacion_matrices(A,B)
print("Tiempo : {}".format(time.time() - t))

Tiempo : 0.11263895034790039


In [7]:
# Para separarlo en Threads particione la matriz premultiplicativa en 4 submatrices con la misma
# cantidad de columnas cada una. Para que cada submatriz hiciera la operaciones por su cuenta.
def obtencion_submatrices(A):
    if len(A) % 4 == 0:
        k = len(A) // 4
        submatrices = [A[i:i+k] for i in range(0,len(A),k)]
    else:
        k = len(A) // 4
        residuo = len(A) - k*4
        sin_residuo = len(A) - residuo
        submatrices = [A[i:i+k] for i in range(0, sin_residuo, k)]
        submatrices.append(A[sin_residuo:])
    return submatrices

In [8]:
submatrices_A = obtencion_submatrices(A)
submatrices_C = []

In [9]:
def multiplicacion_matrices_paralela(B, i):
    
    global submatrices_A,submatrices_C 
    
    A = submatrices_A[i]
    C = []
    
    Am = len(A) 
    An = len(A[0])
    
    Bm = len(B)
    Bn = len(B[0])
    
    if An != Bm:
        return False
    
    for k in range(Am):
        fila_C = []
        
        
        for i in range(Bn):
            sumador = 0
            for j in range(Bm):
                sumador += A[k,j] * B[j,i]
            
            fila_C.append(sumador)
        C.append(fila_C)
    
    submatrices_C.append(C)

In [10]:
candado  = th.Lock()

t = time.time()
# Creo los hilo que utilizaré para multiplicar cada matriz

hilos = [th.Thread(target = multiplicacion_matrices_paralela, args = (B, i)) for i in range(len(submatrices_A))]

for h in hilos:
    h.start()
    
for h in hilos:
    h.join()
    
print("Tiempo : {}".format(time.time() - t))

Tiempo : 0.12643814086914062


In [11]:
# Imprimo las matrices para verificar que son las mismas
for c in submatrices_C:
    for cc in c:
        print()
        print(cc)


[1179, 1302, 1183, 883, 1273, 1249, 1107, 1180, 1340, 1107, 927, 1193, 916, 1092, 1227, 1334, 1327, 1025, 1150, 972, 1181, 1260, 1206, 984, 1331, 901, 1195, 1262, 1298, 1148, 1101, 1036, 977, 967, 1253, 893, 959, 1149, 1270, 1090, 1030, 1190, 937, 1266, 1390, 1208, 1214, 1156, 1123, 929]

[1097, 1219, 1122, 882, 1299, 1150, 1066, 1148, 1272, 1063, 973, 1065, 894, 1065, 1157, 1189, 1225, 956, 1073, 1044, 1177, 1154, 1220, 964, 1149, 876, 969, 1234, 1183, 1089, 1067, 1063, 1009, 980, 1174, 896, 883, 1074, 1229, 1043, 915, 1068, 885, 1182, 1384, 1033, 1141, 1047, 1070, 882]

[1063, 1113, 1159, 781, 1058, 1126, 928, 1017, 1240, 950, 822, 1018, 838, 1015, 1129, 1061, 1171, 979, 1029, 995, 1012, 1141, 1122, 965, 1171, 796, 1057, 1164, 1047, 945, 881, 893, 771, 854, 1126, 835, 931, 948, 1143, 1065, 867, 999, 927, 1095, 1166, 1064, 1150, 1038, 1019, 897]

[1060, 1052, 1145, 883, 1084, 1128, 887, 992, 1186, 916, 821, 1110, 830, 1007, 1100, 1038, 1071, 1049, 1077, 950, 1135, 1080, 1093, 959, 11

In [12]:
for c in C:
    print()
    print(c)


[856, 1029, 1140, 826, 959, 1064, 900, 916, 1140, 912, 845, 981, 706, 924, 992, 1050, 1027, 934, 977, 915, 1001, 1070, 1117, 906, 1058, 688, 898, 1072, 1051, 1059, 771, 918, 732, 714, 977, 848, 876, 1019, 1239, 1059, 959, 931, 748, 1073, 1235, 978, 1034, 885, 985, 854]

[983, 1070, 1070, 851, 1161, 1073, 957, 1003, 1192, 984, 727, 1088, 834, 995, 1056, 1145, 1069, 1009, 1123, 898, 1111, 1000, 1107, 878, 1161, 853, 1025, 1062, 1090, 1100, 879, 1046, 854, 872, 1103, 788, 919, 1055, 1264, 1045, 939, 969, 931, 1152, 1300, 1060, 1113, 1015, 1108, 929]

[935, 1042, 917, 806, 1039, 960, 953, 911, 1042, 962, 725, 928, 778, 839, 954, 960, 1043, 837, 896, 806, 1011, 1004, 1039, 811, 1030, 760, 860, 1094, 1111, 986, 865, 913, 813, 909, 1048, 859, 801, 936, 1050, 917, 812, 961, 779, 1021, 1333, 1008, 1045, 873, 889, 805]

[858, 969, 1013, 714, 1082, 1096, 845, 955, 1165, 934, 909, 914, 748, 901, 1098, 1030, 1142, 842, 877, 914, 818, 881, 1041, 802, 1069, 684, 978, 1113, 1135, 1001, 807, 932, 818,